----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
**CSCI 493.76 Big Data Technology** <br>
**Project 2** <br>

Tools Used:
- Python
    - Pandas Library
- Juypter NoteBook
- Hetio Dataset

Algorithms:
- MapReduce
- Hashing

Resources Used:
- https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
- ChatGpt for Error Handling

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Questions: 
1. Readin nodes.tsv and edges.tsv

In [10]:
import pandas as pd

# Read-in nodes.tsv and edges.tsv
nodes_df = pd.read_csv('nodes_test.tsv', sep='\t')
edges_df = pd.read_csv('edges_test.tsv', sep='\t')

2. For each compound, compute the number of genes that are BIND (CbG)
to it using MapReduce method. Output results with top 5 number of
genes in a descending order 

In [11]:

# Filter Edges DF to only include binding connections (where metaedge = CbG)
CbG_edges_df = edges_df[edges_df['metaedge'] == 'CbG']

# Filter Nodes DF to only include nodes that are Compounds (where kind = Compound)
compound_nodes = nodes_df[nodes_df['kind'] == 'Compound']

#merge Edges and Nodes using source and id cols in files ()
binding_edges_df = pd.merge(CbG_edges_df, compound_nodes, left_on='source', right_on='id')

#print(binding_edges_df.head(5))

# Map
def map_(row):
    return (row['id'], 1)

mapping = binding_edges_df.apply(map_, axis=1) #returns a series 
#print(mapping)

# make it into a DataFrame, first have to convert into a list 
mapped_df = pd.DataFrame(mapping.tolist(), columns=['id','count'])

# Sort (by compound id)
sorted_df = mapped_df.sort_values('id')

# Reduce, use group_by to get all the rows with the same compund id and get the size. 
result_df = sorted_df.groupby('id').size().reset_index(name='gene_count')
#print(result_df)


# Sort the DataFrame by 'count' in descending order
sorted_result_df_2 = result_df.sort_values('gene_count', ascending=False)

print("Compounds with Top Number of Genes: ")

# Print the top 5 pairs
for index, row in sorted_result_df_2.head(5).iterrows():
    print(f"({row['id']}, {row['gene_count']})")


Compounds with Top Number of Genes: 
(Compound::DB01268, 132)
(Compound::DB06616, 104)
(Compound::DB08865, 85)
(Compound::DB01254, 64)
(Compound::DB00331, 56)


3. For each DISEASE, compute the number of GENE(s) that are UPREGULATES (DuG) using MapReduce method. Output results with the
top 5 number of GENE(s) in a descending order.

In [6]:
import pandas as pd

# Read nodes and edges data
nodes_df = pd.read_csv('nodes_test.tsv', sep='\t', names=['id', 'name', 'kind'])
edges_df = pd.read_csv('edges_test.tsv', sep='\t', names=['source', 'metaedge', 'target'])

# Filter nodes for kind 'Disease'
disease_nodes = nodes_df[nodes_df['kind'] == 'Disease']

# Filter edges for metaedge 'DuG'
dug_edges = edges_df[edges_df['metaedge'] == 'DuG']


#Merge to match nodes with corresponding Edges, left join keeps all the rows from the left table, returns only the matches from the right table
merged_df = pd.merge(disease_nodes, dug_edges, left_on='id', right_on='source', how='left')

#Map
def map_(row):
    return (row['id'], row['target'], 1)

mapping = merged_df.apply(map_, axis=1)
mapped_df = pd.DataFrame(mapping.tolist(), columns=['disease_id', 'gene_id', 'count'])

# Sort
sorted_df = mapped_df.sort_values('disease_id')

# Reduce, rest index 
result_df = sorted_df.groupby('disease_id')['gene_id'].nunique().reset_index(name='gene_count')

# Sort the DataFrame by 'gene_count' in descending order
sorted_result_df_3 = result_df.sort_values('gene_count', ascending=False)

print("Diseases with Top Number of Genes: ")

# Print the top 5 pairs
for index, row in sorted_result_df_3.head(5).iterrows():
    print(f"({row['disease_id']}, {row['gene_count']})")

Diseases with Top Number of Genes: 
(Disease::DOID:0050156, 250)
(Disease::DOID:1612, 250)
(Disease::DOID:12365, 250)
(Disease::DOID:7148, 250)
(Disease::DOID:7693, 250)


5. For item number 2 to 4 above, compute the hash tables using mid-square
method (with r = 3, 4) OR Folding Method (digit-size = 2 and 3). Experiment with 10 hash tables for the selected method. Which method
(which digit-size?, r = 3 or r = 4) require least number of storage? Use
sys.getsizeof(.) to find the size of the hash tables (10) with its associated
link list(s).

In [12]:
from collections import defaultdict
import pandas as pd
import math
import sys


# Filter for only binding connections (CbG) and compounds
CbG_edges_df = edges_df[edges_df['metaedge'] == 'CbG']
compound_nodes = nodes_df[nodes_df['kind'] == 'Compound']

# Merge to combine edges and nodes data
binding_edges_df = pd.merge(CbG_edges_df, compound_nodes, left_on='source', right_on='id')

# Total number of hash tables for each value of r
num_tables_r3 = 5
num_tables_r4 = 5

# Initialize the 10 hash tables for r=3 and r=4
hash_tables_r3 = [defaultdict(list) for _ in range(num_tables_r3)]
hash_tables_r4 = [defaultdict(list) for _ in range(num_tables_r4)]

# Function to get middle digits with varying start indices
def get_middle_digits(number, r, offset):
    compound_id_ = str(number)  
    start_index = max(0, (len(compound_id_) - r) // 2 + offset)  #found middle point then shifts it based on offset, starting index cant be negative
    return compound_id_[start_index:start_index + r]  # slice string to return specific substring of size r

# Function to hash IDs
def hash_id(id_str, r, offset):
    # Hash the string and take the absolute value to ensure it is positive
    hashed_value = abs(hash(id_str)) 
    squared_value = hashed_value ** 2
    # Apply mid-square method
    return get_middle_digits(squared_value, r, offset)

# Populate hash tables, iterate the df with merged Compound with CbG edges
for index, row in sorted_result_df_2.iterrows():
    compound_id_str = row['id'].split(':')[-1]   
    gene_total = row['gene_count']  

    # Populate r=3 tables with varying offsets
    for offset, table in enumerate(hash_tables_r3):
        middle_digits = hash_id(compound_id_str, 3, offset - len(hash_tables_r3)//2)  # offset = -2, -1, 0, 1, 2
        table[middle_digits].append(gene_total)  # Appending the gene count

    # Populate r=4 tables with varying offsets
    for offset, table in enumerate(hash_tables_r4):
        middle_digits = hash_id(compound_id_str, 4, offset - len(hash_tables_r4)//2)
        table[middle_digits].append(gene_total)  # Appending the gene count

# Calculate sizes
sizes_r3 = [sys.getsizeof(table) for table in hash_tables_r3]
sizes_r4 = [sys.getsizeof(table) for table in hash_tables_r4]

# Output sizes
print("Sizes for r=3 tables:", sizes_r3)
print("Sizes for r=4 tables:", sizes_r4)

print("Contents of r=3 hash tables:")
for i, table in enumerate(hash_tables_r3):
   print(f"Table {i+1}:", dict(table))

# Directly print hash tables for r=4
print("Contents of r=4 hash tables:")
for i, table in enumerate(hash_tables_r4):
   print(f"Table {i+1}:", dict(table))



Sizes for r=3 tables: [26040, 26040, 26040, 26040, 26040]
Sizes for r=4 tables: [26040, 26040, 26040, 26040, 26040]
Contents of r=3 hash tables:
Table 1: {'198': [132, 4], '071': [104, 2], '805': [85, 2], '948': [64], '404': [56, 5], '740': [52, 4], '183': [50, 17, 11], '016': [47, 12, 9], '275': [46], '441': [44], '735': [43, 22, 13, 5, 5], '727': [43, 4, 2], '718': [43, 5], '323': [41, 19], '106': [41], '440': [41], '787': [40, 4, 1], '918': [39], '129': [37, 1, 1], '370': [36, 12], '409': [36], '161': [36, 1], '295': [36], '699': [34], '189': [34], '810': [34, 8], '865': [34, 3], '195': [33, 21, 2], '038': [32, 11, 1], '889': [32, 11, 4], '253': [32, 8, 5], '278': [32], '219': [32, 21, 5], '463': [31, 23, 6], '644': [31, 1], '830': [31, 4, 3, 2], '417': [31, 11, 2, 1], '143': [31], '124': [31, 12], '798': [31, 19, 14, 6, 6, 3], '657': [31, 16], '803': [30], '581': [30], '793': [30, 2], '339': [29, 4, 3, 2], '469': [29, 6], '388': [29, 2], '633': [29, 8, 3, 1], '035': [29, 20, 1], '3

In [13]:
import sys
from collections import defaultdict

# Define the number of hash tables for each value of r
num_tables_r3 = 5
num_tables_r4 = 5

# Initialize hash tables for r=3 and r=4
hash_tables_r3 = [defaultdict(list) for _ in range(num_tables_r3)]
hash_tables_r4 = [defaultdict(list) for _ in range(num_tables_r4)]

# Function to get middle digits with varying start indices
def get_middle_digits_(number, r, offset):
    number_str = str(number)  # Convert the number to a string
    start_index = max(0, (len(number_str) - r) // 2 + offset)  # Adjust the starting index by offset
    return number_str[start_index:start_index + r]

# Function that gets compound ID
def hash_id(id_str, r, offset):
    # Hash the string and take the absolute value to ensure it is positive
    hashed_value = abs(hash(id_str))
    squared_value = hashed_value ** 2
    # Apply mid-square method to extract middle digits
    return get_middle_digits_(squared_value, r, offset) 

# Populate hash tables with varying start indices
for index, row in sorted_result_df_3.iterrows():
    disease_id_str = row['disease_id'].split(':')[-1]  #get the last part of the broken up string, the part after the : which should be just the numbers of the ID
    gene_total = row['gene_count']

    # Populate r=3 tables with varying offsets
    for offset, table in enumerate(hash_tables_r3):
        middle_digits = hash_id(disease_id_str, 3, offset - len(hash_tables_r3)//2) # offset can be -2, -1, 0, 1, 2
        table[middle_digits].append(gene_total)

    # Populate r=4 tables with varying offsets
    for offset, table in enumerate(hash_tables_r4):
        middle_digits = hash_id(disease_id_str, 4, offset - len(hash_tables_r4)//2)
        table[middle_digits].append(gene_total)

# Calculate sizes
sizes_r3 = [sys.getsizeof(table) for table in hash_tables_r3]
sizes_r4 = [sys.getsizeof(table) for table in hash_tables_r4]

# Output sizes
print("Sizes for r=3 tables:", sizes_r3)
print("Sizes for r=4 tables:", sizes_r4)

#print("Contents of r=3 hash tables:")
#for i, table in enumerate(hash_tables_r3):
#   print(f"Table {i+1}:", dict(table))

# Directly print hash tables for r=4
#print("Contents of r=4 hash tables:")
#for i, table in enumerate(hash_tables_r4):
#   print(f"Table {i+1}:", dict(table))


Sizes for r=3 tables: [3336, 3336, 3336, 3336, 3336]
Sizes for r=4 tables: [3336, 3336, 3336, 3336, 3336]
